# Data Augmentation
Train data에서 Context 문장을 random shuffle 해서 answer의 index 위치를 변경하는 방법으로 Augmentation 진행

In [2]:
import random
import pandas as pd 
from datasets import load_from_disk

In [3]:
# dataset
dataset = load_from_disk("../data/train_dataset/")

train_dataset = pd.DataFrame(dataset['train'])
val_dataset = pd.DataFrame(dataset['validation'])

/opt/conda/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [84]:
train_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [4]:
# answers 데이터 분리
answer_start, text = [], []

for i in range(len(train_dataset)):
    df = train_dataset.iloc[i]
    answer_start.append(df['answers']['answer_start'][0])
    answer = df['answers']['text'][0]
    text.append(answer)


train_df = pd.DataFrame({'title': train_dataset['title'], 
                        'context' : train_dataset['context'], 
                        'question': train_dataset['question'], 
                        'id': train_dataset['id'],
                        'answer': text,
                        'answer_start': answer_start, 
                        'document_id': train_dataset['document_id'], 
                        '__index_level_0__': train_dataset['__index_level_0__'],})

In [5]:
# context 데이터 섞기, 인덱스 변경
new_context = []
new_start_index = []
new_answer = [] # {'answer_start': [235], 'text': ['하원']}	

for i in range(len(train_df)):
    add_context = train_df.iloc[i].context[:train_df.iloc[i].answer_start] + '#' + train_df.iloc[i].context[train_df.iloc[i].answer_start:]
    lst = add_context.split('.')
    random.shuffle(lst)
    random.shuffle(lst)

    random_context = '. '.join(lst)
    num = random_context.index('#')
    random_context = random_context.replace('#', '')

    new_context.append(random_context)
    new_start_index.append(num)
    new_answer.append({'answer_start': [num], 'text': [train_df.iloc[i].answer]})

In [6]:
new_train_df = pd.DataFrame({'title': train_df['title'],
                            'context': new_context,
                            'question': train_df['question'],
                            'id': train_df['id'],
                            'answers': new_answer,
                            'document_id': train_df['document_id'],
                            '__index_level_0__': train_df['__index_level_0__'],})
new_train_df

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 ...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [554], 'text': ['하원']}",18293,42
1,인사조직관리,"25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 ...",현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [355], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,”라고 후대의 황제에게도 이를 훈계하였다. 그럼에도 황제인 그의 생활용품들은 사치...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [122], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [558], 'text': ['중국']}",34146,992
4,계사명 사리구,\n\n동아대학교박물관의 계사명 사리구는 정확한 제작연대는 알 수 없지만 명문 등을...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [169], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
3947,이오 (위성),) 지질구조상으로 정형화된 윤곽이 특별히 없이 고립된 구조처럼 보이는데 이는 지구의...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,"{'answer_start': [962], 'text': ['지질 구조']}",8275,3445
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,"{'answer_start': [482], 'text': ['뉴저지']}",37764,450
3949,빈 필하모니 관현악단,1920년대부터는 잘츠부르크 축제의 상주 관현악단으로도 참가하기 시작해 지금까지 ...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,"{'answer_start': [1143], 'text': ['칼 에케르트']}",12730,2252
3950,드래곤,영국에서는 성 게오르기우스의 상징으로 십자가가 더해진 그림으로 표현된다. \n\n...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,"{'answer_start': [643], 'text': ['슬라브 신화']}",10924,2595


In [7]:
before = pd.read_csv('train_dataset_shuffle.csv')

In [8]:
new_train_dataset = pd.concat([train_dataset, before, new_train_df])

In [102]:
print(f'augmentation BEFORE train dataset count: {len(train_dataset)}')
print(f'augmentation AFTER train dataset count: {len(new_train_df)}')
print(f'augmentation train dataset TOTAL: {len(new_train_dataset)}') # shuffle 데이터 1

augmentation BEFORE train dataset count: 3952
augmentation AFTER train dataset count: 3952
augmentation train dataset TOTAL: 7904


In [9]:
print(f'augmentation train dataset TOTAL: {len(new_train_dataset)}') ## shuffle 데이터 2

augmentation train dataset TOTAL: 15808


In [11]:
new_train_dataset.to_csv('train_dataset_random_aug.csv', index=False)

## ETC.
새로 만든 데이터셋의 `answers` 타입이 `dict`가 아니라 `str` 형태로 불러와져서 `train.py`에서 오류 발생
```python
from ast import literal_eval

answers = literal_eval(answers) # train.py 160번째줄 추가
```     
위 방법으로 해결

# Result.

### baseline result =>     EM: 34.1700   F1: 47.4100

1. shuffle data 3,952 + train data 3,952 = total 7,904

    제출결과 ⇒ EM: 32.9200   F1: 43.7200

2. shuffle data 11,856 + train data 3,952 = total 15,808

    제출결과 ⇒ EM: 32.9200   F1: 43.7200


👉 baseline 대비 점수가 떨어짐 -> answer의 위치가 변경되는 것이 성능에 영향을 주지 않는다는 것을 알 수 있음 

👉 데이터 양을 3배로 늘려서 실험해보았을 때도 차이가 나타나지않았기 때문에 같은 데이터 안에서 변경하는것은 큰 의미가 없다는 것을 확인 할 수 있었음. 